# OLS, Testing

This notebook estimates a linear regression and tests various hypotheses using standard errors assuming (a) iid residuals (Gauss-Markov assumptions); (b) heteroskedasticity (White); (c) autocorrelation and heteroskedasticity (Newey-West).

## Load Packages and Extra Functions

In [1]:
using Dates, DelimitedFiles, Statistics, LinearAlgebra, Distributions

include("jlFiles/printmat.jl")
include("jlFiles/OlsFn.jl")         #functions for OLS

OlsFn

## Loading Data

In [2]:
x = readdlm("Data/FFmFactorsPs.csv",',',skipstart=1)

                #yearmonth, market, small minus big, high minus low
(ym,Rme,RSMB,RHML) = (x[:,1],x[:,2]/100,x[:,3]/100,x[:,4]/100) 
x = nothing

printlnPs("Sample size:",size(Rme))

Sample size:       388


## OLS (assuming iid residuals)

In [3]:
Y = Rme
T = size(Y,1)
X = [ones(T) RSMB RHML]

(b,u,_,V,R2a) = OlsGMFn(Y,X)

printblue("OLS Results (assuming iid residuals)\n")
rowNames = ["c","SMB","HML"]
printTable([b sqrt.(diag(V))],["b","std_iid"],rowNames)

OLS Results (assuming iid residuals)

            b   std_iid
c       0.007     0.002
SMB     0.217     0.073
HML    -0.429     0.074



# Testing a Joint Hypothesis

Since the estimator $\hat{\beta}_{_{k\times1}}$ satisfies

$
\hat{\beta}-\beta_{0} \sim N(0,V_{k\times k})  ,
$

we can easily apply various tests. To test a joint linear hypothesis of the
form

$
\gamma_{J\times1}=R\beta-q,
$

use the test

$
(R\beta-q)^{\prime}\Lambda ^{-1}(R\beta
-q)\overset{d}{\rightarrow}\chi_{J}^{2} \: \text{, where } \: \Lambda=RVR^{\prime}.
$

Clearly, this approach is valid also when $V$ has been estimated by one of the other methods (allowing for heteroskedasticity and/or autocorrelation).

In [4]:
R = [0 1 0;               #testing if b(2)=0 and b(3)=0
     0 0 1]
q = [0;0]
Γ = R*V*R'
test_stat = (R*b-q)'inv(Γ)*(R*b-q)

printblue("Testing Rb = a:")
println("test-statistic and 10% critical value of chi-square(2)")
printmat([test_stat quantile(Chisq(2),0.9)])

Testing Rb = a:
test-statistic and 10% critical value of chi-square(2)
    60.010     4.605



# Distribution of OLS Estimates without Gauss-Markov


The distribution of the OLS estimates is (typically)

$
(\hat{\beta}-\beta_{0})\overset{d}{\rightarrow}N(0,V)
\: \text{ where } \: V=S_{xx}^{-1} S S_{xx}^{-1}
$

and where $S$ is the covariance matrix of $\sum_{t=1}^{T}u_{t}x_{t}$.

*When* the Gauss-Markov assumptions do hold, then $S$ can be simplified as $S=S_{xx}\sigma^2$, where $\sigma^2$ is the variance of $u_t$. Clearly, this means that $V$ can be written $V=S_{xx}^{-1}\sigma^2$. 

In contrast, with heteroskedasticity and/or autocorrrelation $S$ must be estimated differently.

# White's Covariance Matrix

In [5]:
Sxx = X'X

S_iid   = Sxx*var(u)                    #iid, Gauss-Markov
V_iid   = inv(Sxx)'S_iid*inv(Sxx)     
std_iid = sqrt.(diag(V_iid))

S_W   = (X.*u)'*(X.*u)                  #sam as Sum[x_t*x_t'*u_t^2,t=1:T]
V_W   = inv(Sxx)'S_W*inv(Sxx)           #Cov(b), White
std_W = sqrt.(diag(V_W))

printblue("Standard errors from different methods:")
xx = [b std_iid std_W]
printTable(xx,["b","std_iid","std_White"],rowNames,width=12)

Standard errors from different methods:
              b     std_iid   std_White
c         0.007       0.002       0.002
SMB       0.217       0.073       0.113
HML      -0.429       0.074       0.097



# Newey-West's Covariance Matrix

Let $g_t$ be a $K$-vector of data.

To calculate the Newey-West covariance matrix, we first need the
autocovariance matrices $\Gamma_{s}=\text{Cov}(g_{t},g_{t-s})  $, 
which is estimated as 
$ 
\sum_{t=s+1}^{T} (g_{t}-\bar{g})(g_{t-s}-\bar{g})^{\prime}/T.
$

Then we form a linear
combination (with tent-shaped weights) of those autocovariance matrices (from
lag $-m$ to $m$), or equivalently

$
\text{Cov}(\sqrt{T}\bar{g})  = 
\Gamma_{0} + \sum_{s=1}^{m}( 1-\frac{s}{m+1})  
(\Gamma_{s}+\Gamma_{s}^{\prime}).
$

Notice that this is the covariance matrix of $\frac{1}{\sqrt{T}}\sum_{t=1}^T g_t.$ 

If you want covariance matrix of $\frac{1}{T}\sum_{t=1}^T g_t$, then *divide* the result by $T$. 

Conversely, if you want the covariance matrix of $\sum_{t=1}^T g_t$, then *multiply* the result by $T$.

In [6]:
"""
    NWFn(g0,m=0)

Calculates covariance matrix of sqrt(T)*sample average.

# Usage
S = NWFn(g0,m)

# Input
- `g0::Array`: Txq array of q moment conditions
- `m:int`: scalar, number of lags to use

# Output
- `S::Array`: qxq covariance matrix

"""
function NWFn(g0,m=0)

  T = size(g0,1)                    #g0 is Txq
  m = min(m,T-1)                    #number of lags

  g = g0 .- mean(g0,dims=1)         #normalizing to Eg=0

  S = g'g/T                         #(qxT)*(Txq)
  for s = 1:m
    Γ_s = g[s+1:T,:]'g[1:T-s,:]/T   #same as Sum[g_t*g_{t-s}',t=s+1,T]
    S   = S  +  (1 - s/(m+1))*(Γ_s + Γ_s')
  end

  return S

end

NWFn

In [7]:
S_NW   = NWFn(X.*u,2)*T          #Newey-West covariance matrix, 2 lags, notice *T 
V_NW   = inv(Sxx)'S_NW*inv(Sxx)  #Cov(b), Newey-West
std_NW = sqrt.(diag(V_NW))

printblue("Standard errors from different methods:")
xx = [b std_iid std_W std_NW]
printTable(xx,["b","std_iid","std_White","std_NW"],rowNames,width=12)

Standard errors from different methods:
              b     std_iid   std_White      std_NW
c         0.007       0.002       0.002       0.002
SMB       0.217       0.073       0.113       0.129
HML      -0.429       0.074       0.097       0.118

